# References

1. https://realpython.com/numpy-scipy-pandas-correlation-python/#pearson-correlation-numpy-and-scipy-implementation
2. https://blog.smartcat.io/2016/recommender-systems-matrix-operations-for-fast-calculation-of-similarities/
3. https://deepgraph.readthedocs.io/en/latest/what_is_deepgraph.html
4. https://stackoverflow.com/questions/1388818/how-can-i-compare-two-lists-in-python-and-return-matches --- Simiarities between two lists.
5. https://stats.stackexchange.com/questions/269834/how-to-calculate-one-number-from-pearson-correlation-distance-of-more-than-two-v -- Pearson correlation Distance


In [2]:
#load dependencies / mount drive
import numpy as np
import os
import csv 
import json
import math
import time
import scipy

from google.colab import drive
drive.mount('/content/drive')

%cd /content/
%mkdir training

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content
mkdir: cannot create directory ‘training’: File exists


In [3]:
# Copy Given files to local instance for experiements

source_1 ='/content/drive/My\ Drive/Graphika/CF_docl_matrix.json'
source_2 = '/content/drive/My\ Drive/Graphika/AB_docl_matrix.json'

dest_1_sm = 'CF_docl_matrix.json'
dest_2_bg = 'AB_docl_matrix.json'

!cp {source_1} {dest_1_sm}
!cp {source_2} {dest_2_bg}

In [4]:
# Load Datasets to begin work.

with open(dest_1_sm) as sm_file:
  small_data = json.load(sm_file)

with open(dest_2_bg) as bg_file:
  big_data = json.load(bg_file)

# select which dataset to use
data = small_data

# Explore Dataset a little 
Ignore this section for the most part as most of this methodology was wrong. I just left it in here to show the thought process.

In [160]:
min_key = min(data, key=data.get)
max_key = max(data, key=data.get)

vals = data.values()

min_value , max_value = min(min(vals)) , max(max(vals))

# Free Up Ram
del(vals)

# Print results
print("Min Key   : ", min_key)
print("Max Key   : ", max_key)
print("Min Value : ", min_value)
print("Max Value : ", max_value)

Min Key   :  361947
Max Key   :  90258
Min Value :  548
Max Value :  458812


In [161]:
print(data[min_key])
print(len(data[min_key]))

[548, 152169, 190255, 222175, 124431, 86197, 36805, 50957, 428532, 203431, 9081, 44897, 271469, 328841, 45201, 197238, 64553, 235978, 143434, 373893, 134996, 128647, 213559, 91382, 170726, 301005, 313223, 226829, 49963, 20219, 118275, 260609, 165826, 126117, 69598, 121716, 446748, 115577, 145914, 254245, 330779, 417329, 89733, 147882, 37007, 125720, 310983, 434592, 61823, 402703, 390006, 30015, 291370, 186187, 31673, 431201, 191107, 201580]
58


In [162]:
print(data[max_key])
print(len(data[max_key]))

[458812, 103526, 108271, 338600, 145010, 119980, 43725, 151104, 247248, 80336, 149168, 324791, 153145, 101581, 231770, 371982, 450051, 88236, 426575, 384434, 407624, 337672, 250094, 180586, 171408, 144318, 209298, 331484, 24570, 260580, 230994, 229740, 257640, 222347, 208852, 132673, 55181, 445236, 390348, 176446, 427758, 348430, 82322, 69616, 61461, 340975, 197573, 424561, 33830, 141336, 357942, 40054, 190337, 243554, 269538, 53049, 167878, 102988, 242488, 278618, 83505, 2206, 66514, 218112, 175427, 168202, 372502, 73856, 375436, 205673, 218309, 243819, 293466, 103239, 268483, 260315, 246984, 76315, 345706, 440928, 451870, 216243, 166192, 152557, 286155, 322254, 288843, 427495, 40790, 93466, 307878, 406335, 290202, 382256, 97926, 386041, 324357, 18556, 203951, 362509, 35646, 316510, 276244, 140288, 30107, 208954, 164833, 218765, 17663, 149559, 389154, 91228, 200594, 145873, 211774, 197904, 288128, 41732, 288865, 118328, 267990, 93530, 163575, 238745, 54266, 262547, 282179, 199282, 296

In [163]:
# DOES NOT WORK AS THE ARRAYS ARE OF DIFFERENT LENGTH
r, m = scipy.stats.pearsonr(data[min_key], data[max_key]) 

ValueError: ignored

#Re-Factor dataset using sets and generate length of feature list to optimize calculations of PCC.

Varibles for Big O calculations:
1. n = # of rows (index_values).
2. m = len([list of non-empty column names])

Note: length of m varies.

In [5]:
id_list = [] # List of Anonymized Twitter ID's
data_set = {} # Set version of original data
max_feature = float('-inf') # maximum value in feature column
min_feature = float('inf') # minimum value in feature column - Might not  need this

# O(n * m)
for key in data: #O(n)
  id_list.append(key) #O(1)
  data_set[key] = set(data[key]) #O(m)
  max_feature = max(max_feature, max(data_set[key])) #O(m)
  min_feature = min(min_feature, min(data_set[key])) #O(m)

# Since min_feature == 0, add 1 to max_feature
max_feature+= 1

print("Total # of IDs:    ", len(id_list))
print("Feature Value Min: ", min_feature)
print("Feature Value Max: ", max_feature)


Total # of IDs:     1000
Feature Value Min:  0
Feature Value Max:  459316


In [10]:
def create_PCC_matrix(max_feature, id_list, data_set):
  adjacency_matrix = np.zeros((len(id_list), len(id_list)))
  adjacency_list = []

  #O(n * max(len(m)))
  for x in range(0,len(id_list)):
    for y in range(x+1, len(id_list)):
      # sum of all x is the same as the length of feature set
      sig_x = len(data_set[id_list[x]])

      # average of all x values
      avg_x = sig_x / max_feature

      # sum of all y is the same as the length of feature set
      sig_y = len(data_set[id_list[y]])
      avg_y = sig_y / max_feature
    
      sen_a = data_set[id_list[x]].intersection(data_set[id_list[y]]) # x = 1, y = 1  -- O(set(a) + set(b))
      sen_a_total = len(sen_a) * ((1-avg_x) * (1-avg_y))
      sen_a_sd_x = len(sen_a) * ((1-avg_x)**2)
      sen_a_sd_y = len(sen_a) * ((1-avg_y)**2)

      sen_b = data_set[id_list[x]].difference(data_set[id_list[y]])  # x = 1, y = 0
      sen_b_total = len(sen_b) * ((1-avg_x) * (-avg_y))
      sen_b_sd_x = len(sen_b) * ((1-avg_x)**2)
      sen_b_sd_y = len(sen_b) * ((-avg_y)**2)


      sen_c = data_set[id_list[y]].difference(data_set[id_list[x]]) # x = 0, y = 1
      sen_c_total = len(sen_c) * ((-avg_x) * (1-avg_y))
      sen_c_sd_x = len(sen_c) * ((-avg_x)**2)
      sen_c_sd_y = len(sen_c) * ((1-avg_y)**2)

      sen_d = (max_feature - len(data_set[id_list[x]].union(data_set[id_list[y]])))
      sen_d_total = sen_d * (avg_x * avg_y)
      sen_d_sd_x = sen_d * (avg_x**2)
      sen_d_sd_y = sen_d * (avg_y**2)

      numerator = sen_a_total + sen_b_total + sen_c_total + sen_d_total
      denominator = math.sqrt((sen_a_sd_x + sen_b_sd_x + sen_c_sd_x + sen_d_sd_x) *
                              (sen_a_sd_y + sen_b_sd_y + sen_c_sd_y + sen_d_sd_y))

      r_val = 1 - (numerator / denominator)

      adjacency_matrix[x][y] = r_val
      adjacency_list.append( [id_list[x], id_list[y], r_val ])

  return adjacency_list , adjacency_matrix

adjacency_list, adjacency_matrix = create_PCC_matrix(max_feature, id_list, data_set)

#TEST AREA DEFINITIONS

Method 'test_SLOW_version' defines a block of code that iterates through all features (max_features) and calculate the Pearson correlation distance with a O(max_features) runtime. For the small dataset max_features == 459315

In [7]:
def test_SLOW_version(data_set, id_list, x: int, y: int, callback=False) -> int:
  sig_x = len(data_set[id_list[x]])
  avg_x = sig_x / max_feature

  sig_y = len(data_set[id_list[y]])
  avg_y = sig_y / max_feature

  top = 0
  bottom_x = 0
  bottom_y = 0

  for i in range(0,max_feature): #O(max_feature) -- Very Expensive
    x_val = int(1 if i in data_set[id_list[x]] else 0)   #O(1) avg lookup for set 'in'
    y_val = int(1 if i in data_set[id_list[y]] else 0)   #O(1) avg lookup for set 'in'
    top += ((x_val - avg_x) * (y_val - avg_y))
    bottom_x += (x_val - avg_x)**2
    bottom_y += (y_val - avg_y)**2
  #print(top, " ", str((math.sqrt(bottom_x * bottom_y))))
  r_val = 1 - (top / (math.sqrt(bottom_x * bottom_y)))

  if callback == True:
    print("max_feature                       : ", max_feature)
    print("ID of first element in ID List    : ", id_list[x])
    print("ID of second element in ID List   : ", id_list[y])
    print("Pearson Coorelation Distance (1-r): ", r_val)

  return r_val

Method 'test_FAST_version' defines a block of code that utilizes sets and domain knowledge of sparse matricies to reduce runtime to O( len(set(x)) + len(set(y)).

Where len(set(x)) is the length of the feature list of the first ID

Where len(set(y)) is the length of the feature list of the second ID

The above is drastically smaller and faster 'test_SLOW_version'

In [8]:
def test_FAST_version(set_a, set_b, x: int, y:int, callback=False) -> int:
  # length of set_a is the same as the sum of all 'x'
  sig_x = len(set_a)

  # mean of all set_a values = sum of all x / total feature count
  avg_x = sig_x / max_feature

  # do same calculations for 'y'
  sig_y = len(set_b)
  avg_y = sig_y / max_feature

  # Senario A: x = 1, y = 1  
  # -- Intersection Time Complexity (Avg): O(min(len(a), len(b)))
  sen_a = set_a.intersection(set_b) 
  numerator = len(sen_a) * ((1-avg_x) * (1-avg_y))
  denom_x = len(sen_a) * ((1-avg_x)**2)
  denom_y = len(sen_a) * ((1-avg_y)**2)

  # Senario B: x = 1, y = 0  
  # -- Difference Time Complexity (Avg): O(len(a))
  sen_b = set_a.difference(set_b)  
  numerator += len(sen_b) * ((1-avg_x) * (-avg_y))
  denom_x += len(sen_b) * ((1-avg_x)**2)
  denom_y += len(sen_b) * ((-avg_y)**2)

  # Senario C: x = 0, y = 1  
  # -- Difference Time Complexity (Avg): O(len(b))
  sen_c = set_b.difference(set_a)
  numerator += len(sen_c) * ((-avg_x) * (1-avg_y))
  denom_x += len(sen_c) * ((-avg_x)**2)
  denom_y += len(sen_c) * ((1-avg_y)**2)

  # Senario D: x = 0, y = 0  
  # -- Union Time Complexity (Avg): O(len(a) + len(b))
  sen_d = (max_feature - len(set_a.union(set_b)))
  numerator += sen_d * (avg_x * avg_y)
  denom_x += sen_d * (avg_x**2)
  denom_y += sen_d * (avg_y**2)

  denominator = math.sqrt(denom_x * denom_y)

  r_val = 1 - (numerator / denominator)
  #print(numerator, " ", denominator)
  if callback == True:
    print("max_feature                       : ", max_feature)
    print("ID of first element in ID List    : ", id_list[x])
    print("ID of second element in ID List   : ", id_list[y])
    print("Pearson Coorelation Distance (1-r): ", r_val)
  
  return r_val

#TEST AREA

In [12]:
x_range = 25
callback = False
fast_exec_time = []
slow_exec_time = []
error = []

for x in range(0,x_range):
  for y in range(x+1, x_range):
    start_time = time.time()
    a = test_SLOW_version(data_set, id_list, x, y, callback)
    slow_exec_time.append(time.time() - start_time)
    start_time = time.time()
    b = test_FAST_version(data_set[id_list[x]], data_set[id_list[y]], x, y, callback)
    fast_exec_time.append(time.time() - start_time)
    error.append(abs(a-b))

avg_error = sum(error) / len(error)
avg_fast = sum(fast_exec_time) / len(fast_exec_time)
avg_slow = sum(slow_exec_time) / len(slow_exec_time)

In [13]:
print("Average SLOW method time (ms): ", str(avg_slow*1000))
print("Average FAST method time (ms): ", str(avg_fast*1000))
print("")
print("Total   SLOW method time (ms): ", str(sum(slow_exec_time)*1000))
print("Total   FAST method time (ms): ", str(sum(fast_exec_time)*1000))
print("")
print("Average Error                : ", avg_error)

Average SLOW method time (ms):  329.4571097691854
Average FAST method time (ms):  0.19491275151570636

Total   SLOW method time (ms):  98837.13293075562
Total   FAST method time (ms):  58.473825454711914

Average Error                :  6.93012314201269e-14


In [11]:
with open('adj_matrix.csv', 'w') as fh:
  writer = csv.writer(fh, delimiter=',')
  writer.writerow(id_list)
  for x in adjacency_matrix:
    writer.writerow(x)

csv_columns = ['id_1','id_2', 'PCD']
with open('adj_list.csv', 'w') as fh:
  writer = csv.writer(fh, delimiter=',')
  writer.writerow(csv_columns)
  for x in adjacency_list:
    writer.writerow(x)